In [ ]:
# 📁 Importar librerías necesarias
import os
import torch
import ffmpeg
import numpy as np
from transformers import VideoMAEFeatureExtractor, VideoMAEModel
from decord import VideoReader, cpu

In [ ]:
# 🛠️ Funciones auxiliares
def get_video_clips(video_path, clip_duration):
    vr = VideoReader(video_path, ctx=cpu(0))
    num_frames = len(vr)
    fps = vr.get_avg_fps()
    clip_len = int(fps * clip_duration)
    clips = []
    for start in range(0, num_frames, clip_len):
        end = min(start + clip_len, num_frames)
        clip = vr.get_batch(list(range(start, end)))
        clips.append(clip)
    return clips

def extract_embeddings(clips, model, processor):
    embeddings = []
    for clip in clips:
        inputs = processor(list(clip.asnumpy()), return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze(0).numpy())
    return np.array(embeddings)

def detect_scene_changes(embeddings, threshold=0.75):
    scene_changes = [0]
    for i in range(1, len(embeddings)):
        sim = np.dot(embeddings[i-1], embeddings[i]) / (
            np.linalg.norm(embeddings[i-1]) * np.linalg.norm(embeddings[i])
        )
        if sim < threshold:
            scene_changes.append(i)
    return scene_changes

def split_video(input_path, scene_changes, clip_duration, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for idx, scene_idx in enumerate(scene_changes):
        start = scene_idx * clip_duration
        output = os.path.join(output_dir, f"scene_{idx+1}.mp4")
        ffmpeg.input(input_path, ss=start, t=clip_duration).output(output, codec='copy').run(overwrite_output=True)

In [ ]:
# 🚀 Ejecutar flujo completo
clip_duration = 1.0
threshold = 0.75
video_path = "video.mp4"

if not os.path.exists(video_path):
    from google.colab import files
    print("⚠️ Archivo no encontrado. Sube tu video ahora...")
    uploaded = files.upload()

print("[INFO] Cargando modelo...")
processor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

print("[INFO] Extrayendo clips...")
clips = get_video_clips(video_path, clip_duration)

print("[INFO] Generando embeddings...")
embeddings = extract_embeddings(clips, model, processor)

print("[INFO] Detectando cortes de escena...")
scene_changes = detect_scene_changes(embeddings, threshold)
print("Cortes detectados:", scene_changes)

print("[INFO] Cortando video...")
split_video(video_path, scene_changes, clip_duration, "escenas")
print("[LISTO] Escenas generadas en la carpeta 'escenas'")